In [2]:
import os
import json
import pandas as pd
import traceback

In [18]:
from dotenv import load_env()

SyntaxError: invalid syntax (1662801003.py, line 1)

In [ ]:
KEY = os.getenv("OPENAI_API_KEY")

In [12]:
import langchain

In [13]:
import langchain_community

In [14]:
from langchain.chat_models import ChatOpenAI

In [15]:


llm = ChatOpenAI(openai_api_key="OPENAI_API_KEY",model_name="gpt-3.5-turbo",temperature=0.5)